<a href="https://colab.research.google.com/github/nadatanalytics/dataanalyticsproject/blob/main/SA_DEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp
import pulp
!apt-get install -y -qq glpk-utils

In [ ]:
from pulp import GLPK
import pandas as pd # Library to handle data
import numpy as np
from statistics import mean

data = pd.read_csv("/content/FinalUnivDataset.csv")

#creating a list to store the optimal decision variables
weights_matrix = np.empty((7,5))
for k in range(5):  
    model = pulp.LpProblem("University Analysis", pulp.LpMaximize) # Create an LP maximization problem
    m1 = pulp.LpVariable("m1", lowBound=0, upBound=None, cat='Continuous') # Create a variable m1>=0
    m2 = pulp.LpVariable("m2", lowBound=0, upBound=None, cat='Continuous') # Create a variable m2>=0
    n1 = pulp.LpVariable("n1", lowBound=0, upBound=None, cat='Continuous') # Create a variable n1>=0
    n2 = pulp.LpVariable("n2", lowBound=0, upBound=None, cat='Continuous') # Create a variable n2>=0
    n3 = pulp.LpVariable("n3", lowBound=0, upBound=None, cat='Continuous') # Create a variable n3>=0
    n4 = pulp.LpVariable("n4", lowBound=0, upBound=None, cat='Continuous') # Create a variable n3>=0
    n5 = pulp.LpVariable("n5", lowBound=0, upBound=None, cat='Continuous') # Create a variable n3>=0
   
    model += m1*data["Y1"][k] + m2*data["Y2"][k]  #objective function
    
    model += n1*data["X1"][k] + n2*data["X2"][k] + n3*data["X3"][k]+ n4*data["X4"][k] + n5*data["X5"][k] == 1 #first constraint (equality)

    for i in range(5): 
        model += m1*data["Y1"][i] + m2*data["Y2"][i] - n1*data["X1"][i] - n2*data["X2"][i] - n3*data["X3"][i] - n4*data["X4"][i] - n5*data["X5"][i] <= 0
   
    status = model.solve(GLPK(msg=True, options=['--ranges', 'sensitivity.txt'])) 
    print("Model Status:{}".format(pulp.LpStatus[model.status])) 
    


    # Displaying Optimal Decision Variables & Reduced Cost per Variable
    list_weights=[]
    for v in model.variables():
	      print(v.name, "=", v.varValue, "\tReduced Cost =", v.dj)
    for v in model.variables():  
        list_weights=list_weights+[v.varValue]
    for n in range(7):
        value= list_weights[n]
        weights_matrix[n,k] = value
        
    # Displaying Optimal Value of Objective Function	
    print("Objective=", pulp.value(model.objective))
	
    # Displaying Shadow Price & Slack Value per Constraint
    print ("\nSensitivity Analysis\nConstraint\t\tShadow Price\tSlack")
    for name, c in model.constraints.items():
	      print(name, ":", c, "\t", c.pi, "\t\t", c.slack)

Model Status:Optimal
m1 = 1.33156 	Reduced Cost = None
m2 = 0.0 	Reduced Cost = None
n1 = 2.16624e-05 	Reduced Cost = None
n2 = 0.0 	Reduced Cost = None
n3 = 0.00518358 	Reduced Cost = None
n4 = 0.0 	Reduced Cost = None
n5 = 0.125675 	Reduced Cost = None
Objective= 1.00000156

Sensitivity Analysis
Constraint		Shadow Price	Slack
_C1 : 28040.5*n1 + 2*n2 + 3*n3 + 0.905*n4 + 3*n5 = 1.0 	 None 		 None
_C2 : 0.751*m1 + 0.67*m2 - 28040.5*n1 - 2*n2 - 3*n3 - 0.905*n4 - 3*n5 <= -0.0 	 None 		 None
_C3 : 0.955*m1 + 0.771*m2 - 34300.0*n1 - 3*n2 - 5*n3 - 0.822*n4 - 4*n5 <= -0.0 	 None 		 None
_C4 : 0.689*m1 + 0.605*m2 - 22153.5*n1 - 2*n2 - 2*n3 - 0.879*n4 - 5*n5 <= -0.0 	 None 		 None
_C5 : 0.784*m1 + 0.805*m2 - 19637.5*n1 - 2*n2 - 4*n3 - 0.859*n4 - 5*n5 <= -0.0 	 None 		 None
_C6 : 0.627*m1 + 0.395*m2 - 32500.0*n1 - 2*n2 - n3 - 0.781*n4 - n5 <= -0.0 	 None 		 None
Model Status:Optimal
m1 = 1.04712 	Reduced Cost = None
m2 = 0.0 	Reduced Cost = None
n1 = 1.69925e-05 	Reduced Cost = None
n2 = 0.0 	Re

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
efficiency_table =np.empty((5,5))
average_list =[]
for row in range(5):
    row_values=[]
    for column in range(5):
       value = ((weights_matrix[0,column]*data["Y1"][row])+(weights_matrix[1,column]*data["Y2"][row]))/((weights_matrix[2,column]*data["X1"][row])+(weights_matrix[3,column]*data["X2"][row])+(weights_matrix[4,column]*data["X3"][row])+(weights_matrix[5,column]*data["X4"][row])+(weights_matrix[6,column]*data["X5"][row]))
       efficiency_table[row,column]=value
       row_values=row_values+[value]
    average =mean(row_values)
    average_list =average_list+[average]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
average_list_rounded = [ '%.2f' % elem for elem in average_list ]

In [ ]:
uni_list = ['Warwick','Imperial','Edinburgh','Manchester','Durham']

In [ ]:
for uni in uni_list:
    index = uni_list.index(uni)
    print("Cross-efficiency score for",uni,"=",average_list_rounded[index])

Cross-efficiency score for Warwick = 0.87
Cross-efficiency score for Imperial = 0.86
Cross-efficiency score for Edinburgh = 0.71
Cross-efficiency score for Manchester = 0.84
Cross-efficiency score for Durham = 0.96


In [ ]:
display(efficiency_table)
efficiency_table = efficiency_table.round(decimals=2)
display(efficiency_table)

array([[1.00000129, 0.99624871, 0.96062902, 1.00000424, 0.39925663],
       [1.00000124, 0.99999685, 0.9999999 , 0.89879085, 0.38078238],
       [0.82014297, 0.80350933, 0.99999992, 0.72547535, 0.21977722],
       [0.97155744, 0.96000874, 1.00000005, 1.00000367, 0.25008032],
       [1.00000184, 0.99999694, 0.99999971, 0.80671632, 1.0000023 ]])

array([[1.  , 1.  , 0.96, 1.  , 0.4 ],
       [1.  , 1.  , 1.  , 0.9 , 0.38],
       [0.82, 0.8 , 1.  , 0.73, 0.22],
       [0.97, 0.96, 1.  , 1.  , 0.25],
       [1.  , 1.  , 1.  , 0.81, 1.  ]])